<a href="https://colab.research.google.com/github/Mahecoding/PRODIGY_GA_01/blob/main/PRODIGY_GA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This code fine-tunes a pre-trained GPT-2 model on a custom text dataset and then uses Gradio to create a web interface where users can generate text based on prompts they provide.**

In [2]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

**Importing Libraries**

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, TextDataset
import gradio as gr


**Tokenizing and Loading the Dataset**

In [4]:

def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

**Defining the Training Process**

In [5]:

def train(dataset, model, tokenizer):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Masked Language Modeling is not used for GPT-2
    )

    training_args = TrainingArguments(
        output_dir='./results',  # Directory where the model checkpoints will be saved
        overwrite_output_dir=True,  # Overwrite the content of the output directory if it exists
        num_train_epochs=3,  # Number of training epochs (iterations over the dataset)
        per_device_train_batch_size=4,  # Number of samples per batch per device (e.g., per GPU)
        save_steps=10_000,  # Save the model every 10,000 steps
        save_total_limit=2,  # Limit the total number of saved checkpoints to 2
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )
    trainer.train()

    # Save the final trained model
    trainer.save_model()

**Loading the GPT-2 Model and Tokenizer**

In [6]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

**Preparing the Dataset and Training the Model**

In [7]:
dataset = load_dataset('/content/drive/MyDrive/PRODIGY/dataset.txt', tokenizer)
train(dataset, model, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


**Generating Text with the Fine-Tuned Model**

In [8]:
def generate_text(prompt, max_length= 100, num_return_sequences=1):
  input_ids= tokenizer.encode(prompt, return_tensors='pt')
  output = model.generate(
      input_ids,
      max_length=max_length,
      num_return_sequences= num_return_sequences,
      no_repeat_ngram_size=2,
      repetition_penalty=1.5,
      length_penalty=1.0,
      early_stopping=True
  )
  return tokenizer.decode(output[0], skip_special_tokens=True)


**Creating the Gradio Interface**

In [9]:

gr_interface = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="GPT-2 Text Generator",
    description="Generate text using a fine-tuned GPT-2 model."
)


**Test Outside of Gradio**

In [10]:
prompt = "In a faraway kingdom, a wise old king ruled with kindness and wisdom. Describe it."
print(generate_text(prompt))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In a faraway kingdom, a wise old king ruled with kindness and wisdom. Describe it.
The story begins in the village of Kainu Village where there are many different cultures that share their beliefs about life as well from what to do when you're young or dead. The villagers have been taught how to live together peacefully for centuries but now they see an opportunity is becoming more peaceful after years spent living apart on one island surrounded by people who love each other dearly...and all too often


**Launching the Gradio App**

In [11]:
gr_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6b4ff312f2845f80ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
